# Multimodal Classification Training

This notebook creates the Multimodal Network Architecture and trains it for grasp testset1. After training the network weights will be stored in the folder `./dataset/grasp_testset1_logs`

## Dependencies

`Python 3.5.4` is used for development and following packages are required to run the code provided in the notebook:

`pip install googledrivedownloader`<br>
`pip install matplotlib`<br>
`pip install tensorflow-gpu`<br>
`pip install keras`<br>
`pip install numpy`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow==1.15.0
!pip install keras==2.2.4

In [ ]:
import os, csv, time, shutil
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
path="/content/drive/MyDrive/Deep-Multi-Sensory-Object-Categorization"

In [ ]:
def print_image(image, title):
    """Print the image

    :param image: image pixels in list
    :param title: title as string to be printed on top of the image
    """
    plt.imshow(image)
    plt.title(title)
    plt.colorbar()
    plt.show()

def time_taken(start, end):
    """Human readable time between `start` and `end`

    :param start: time.time()
    :param end: time.time()
    :returns: day:hour:minute:second
    """
    time = end-start
    day = time // (24 * 3600)
    time = time % (24 * 3600)
    hour = time // 3600
    time %= 3600
    minutes = time // 60
    time %= 60
    seconds = time
    day_hour_min_sec = str('%02d' % int(day))+":"+str('%02d' % int(hour))+":"+str('%02d' % int(minutes))+":"+str('%02d' % int(seconds))
    
    return day_hour_min_sec

## Video Dataset

In [ ]:
file_0 = path+"/dataset/EMILver1_preprocessed/EMILver1_vi_vgg16fc2_pca19/arr_0.npy"
file_1 = path+"/dataset/EMILver1_preprocessed/EMILver1_vi_vgg16fc2_pca19/arr_1.npy"
file_2 = path+"/dataset/EMILver1_preprocessed/EMILver1_vi_vgg16fc2_pca19/arr_2.npy"
video_frames = np.load(file_0, allow_pickle=True)
action_label = np.load(file_1, allow_pickle=True)
object_label = np.load(file_2, allow_pickle=True)
for i in range(len(video_frames)):
    a01 = video_frames[i]
    while len(a01) < 658:
        a01 = np.concatenate((a01, np.zeros((1, a01.shape[1]))))
    video_frames[i] = a01
video_frames = np.array(list(video_frames))

## Sound Dataset

In [ ]:
file_0 = path+"/dataset/EMILver1_preprocessed/EMILver1_au_features/arr_0.npy"
file_1 = path+"/dataset/EMILver1_preprocessed/EMILver1_au_features/arr_1.npy"
file_2 = path+"/dataset/EMILver1_preprocessed/EMILver1_au_features/arr_2.npy"
audio_frames = np.load(file_0, allow_pickle=True)
action_label = np.load(file_1, allow_pickle=True)
object_label = np.load(file_2, allow_pickle=True)
for i in range(len(audio_frames)):
    a01 = audio_frames[i]
    while len(a01) < 658:
        a01 = np.concatenate((a01, np.zeros((1, a01.shape[1]))))
    audio_frames[i] = a01
audio_frames = np.array(list(audio_frames))

## Haptic Dataset

In [ ]:
file_0 = path+"/dataset/EMILver1_preprocessed/EMILver1_sm_features/arr_0.npy"
file_1 = path+"/dataset/EMILver1_preprocessed/EMILver1_sm_features/arr_1.npy"
file_2 = path+"/dataset/EMILver1_preprocessed/EMILver1_sm_features/arr_2.npy"
haptic_frames = np.load(file_0, allow_pickle=True)
action_label = np.load(file_1, allow_pickle=True)
object_label = np.load(file_2, allow_pickle=True)
for i in range(len(haptic_frames)):
    a01 = haptic_frames[i]
    while len(a01) < 658:
        a01 = np.concatenate((a01, np.zeros((1, a01.shape[1]))))
    haptic_frames[i] = a01
haptic_frames = np.array(list(haptic_frames))

In [ ]:
# one-hot encoding
num_classes = np.nanmax(action_label)+1
action_label_one_hot = np.zeros((len(action_label), num_classes)).astype(int)
for i in range(len(action_label)):
    action_label_one_hot[i, action_label[i]] = 1

num_classes = np.nanmax(object_label)+1
object_label_one_hot = np.zeros((len(object_label), num_classes)).astype(int)
for i in range(len(object_label)):
    object_label_one_hot[i, object_label[i]] = 1

# train-test-split
num_data = len(object_label)
train_id, test_id = train_test_split(np.array(range(num_data)), random_state=0)

## Multimodal Network Hyper-parameters

This network was trained for 300 epochs using Adam optimization with learning rate 1 x $10^{-4}$.

In [ ]:
# Network hyper-parameters
batch = 5
training_epochs = 300
display_step = 1

behavior = "grasp"
testset = "testset1"
folder_name = behavior+'_'+testset
model_path = path+"/dataset/"+folder_name+"_logs/model.ckpt"
logs_path = path+"/dataset/"+folder_name+"_logs/"

# num_classes = category_label_train_one_hot.shape[1]
num_classes = object_label_one_hot.shape[1]

Y = tf.placeholder('float', [None, num_classes], name='LabelData')
print("Y: ", Y)

video_frames_max = 658
video_size = video_frames.shape[2]
video_X = tf.placeholder('float', [None, video_frames_max, video_size], name='InputData')

audio_frames_max = 658
audio_size = audio_frames.shape[2]
audio_keep_prob = tf.placeholder_with_default(1.0, shape=(), name='audio_keep')

haptic_frames_max = 658
haptic_size = haptic_frames.shape[2]
haptic_keep_prob = tf.placeholder_with_default(1.0, shape=(), name='haptic_keep')

In [ ]:
"""
Functions used to define models
"""

haptic_skip_2nd_maxpool = ["grasp", "hold", "low"]

def model(video_data_placeholder):
    with tf.name_scope("Model"):
        # Video
        net = tf.layers.flatten(video_data_placeholder)
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net)
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net)
        video_logits = tf.layers.dense(inputs=net, units=num_classes, activation=tf.nn.relu)
        
        # Audio
        audio_data_placeholder = tf.placeholder('float', [None, audio_frames_max, audio_size], name='audio_InputData')
        net = tf.layers.flatten(audio_data_placeholder)
        # Dense Layer
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net, rate=audio_keep_prob)
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net, rate=audio_keep_prob)
        audio_logits = tf.layers.dense(inputs=net, units=num_classes, activation=tf.nn.relu)
        
        # Haptic
        haptic_data_placeholder = tf.placeholder('float', [None, haptic_frames_max, haptic_size], name='haptic_InputData')
        net = tf.layers.flatten(haptic_data_placeholder)
        # Dense Layer
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net, rate=haptic_keep_prob)
        haptic_logits = tf.layers.dense(inputs=net, units=num_classes, activation=tf.nn.relu)
        
        # Concatenate 
        logits = tf.concat([video_logits, audio_logits, haptic_logits], axis=1)
        logits = tf.nn.relu(logits)
        logits = tf.layers.dense(inputs=logits, units=num_classes)
        
    return logits


def loss(prediction, label_placeholder):
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=label_placeholder))
        # Create a summary to monitor cost tensor
        cost_scalar = tf.summary.scalar("loss", cost)
    return cost, cost_scalar

def training(prediction, label_placeholder):
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(cost)
    return train_op

def evaluate(prediction, Y):
    with tf.name_scope('Accuracy'):
        # Test model
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        # Create a summary to monitor accuracy tensor
        accuracy_scalar = tf.summary.scalar("accuracy", accuracy)
    return accuracy, accuracy_scalar

In [ ]:
"""
Creating the Neural Network
"""

model_dict = {}
prediction = model(video_X)
model_dict["Model"] = prediction

cost, cost_scalar = loss(prediction, Y)
model_dict["Loss"] = cost
model_dict["Loss_scalar"] = cost_scalar

train_op = training(prediction, Y)
model_dict["Optimizer"] = train_op

eval_op, accuracy_scalar = evaluate(prediction, Y)
model_dict["Accuracy"] = eval_op
model_dict["Accuracy_scalar"] = accuracy_scalar

print("model_dict: ", model_dict)

# Initializing the variables
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver(max_to_keep=1)

In [ ]:
if os.path.exists(logs_path):
    shutil.rmtree(logs_path)
    os.makedirs(logs_path)
else:
    os.makedirs(logs_path)

In [ ]:
"""
Writing 'Time', 'Epoch', 'Cost', 'Accuracy' in CSV file
"""

epoch_cost_accuracy = []
epoch_cost_accuracy.append("Time")
epoch_cost_accuracy.append("Epoch")
epoch_cost_accuracy.append("Cost")
epoch_cost_accuracy.append("Accuracy")

with open(logs_path+folder_name+"_data.csv",'w') as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerow(epoch_cost_accuracy)

## Training

In [ ]:
"""## Training"""

# Start Training

start_time = time.time()

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost_list = 0.0
        total_batch = int(len(train_id)/batch)
        
        # Shuffle data
        np.random.shuffle(train_id)
        
        i = 0
        # Loop over all batches
        for start, end in zip(range(0, len(train_id), batch), range(batch, len(train_id)+1, batch)):
            video_input_data, label_data = video_frames[train_id][start:end], object_label_one_hot[train_id][start:end]

            audio_input_data = audio_frames[train_id][start:end]
            audio_X = tf.get_default_graph().get_tensor_by_name("Model/audio_InputData:0")

            haptic_input_data = haptic_frames[train_id][start:end]
            haptic_X = tf.get_default_graph().get_tensor_by_name("Model/haptic_InputData:0")

            _, new_cost, cost_scalar = sess.run(
                [model_dict["Optimizer"], model_dict["Loss"], model_dict["Loss_scalar"]], 
                feed_dict={
                    video_X: video_input_data, 
                    audio_X: audio_input_data, 
                    haptic_X: haptic_input_data, 
                    Y: label_data, 
                    audio_keep_prob: 0.5, 
                    haptic_keep_prob: 0.5
                    }
                )
            # Compute average loss
            avg_cost_list += new_cost/total_batch

            summary_writer.add_summary(cost_scalar, epoch * total_batch + i)
            i += 1
        save_path = saver.save(sess, model_path, epoch)
         
        # Calculate Accuracy
        avg_accuracy_list = 0.0
        total_batch = int(len(test_id)/batch)
        i = 0
        for start, end in zip(range(0, len(test_id), batch), range(batch, len(test_id)+1, batch)):
            video_input_data, label_data = video_frames[test_id][start:end], object_label_one_hot[test_id][start:end]

            audio_input_data = audio_frames[test_id][start:end]
            audio_X = tf.get_default_graph().get_tensor_by_name("Model/audio_InputData:0")

            haptic_input_data = haptic_frames[test_id][start:end]
            haptic_X = tf.get_default_graph().get_tensor_by_name("Model/haptic_InputData:0")

            accuracy, accuracy_scalar = sess.run([model_dict["Accuracy"], model_dict["Accuracy_scalar"]], feed_dict={video_X: video_input_data, audio_X: audio_input_data, haptic_X: haptic_input_data, Y: label_data, audio_keep_prob: 1.0, haptic_keep_prob: 1.0})
            # Compute average accuracy
            avg_accuracy_list += accuracy/total_batch
            summary_writer.add_summary(accuracy_scalar, epoch * total_batch + i)
            i += 1
        
        # Printing current epoch accuracy
        epoch_cost_accuracy = []
        epoch_cost_accuracy.append(time_taken(start_time, time.time()))
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), ", Time: ", time_taken(start_time, time.time()))
            a_string = "Cost - "
            epoch_cost_accuracy.append(epoch+1)
            
            a_string += str(avg_cost_list)
            epoch_cost_accuracy.append(str(avg_cost_list))
            
            a_string = a_string[0:-2]+" --> Accuracy - "
            a_string += str(avg_accuracy_list)
            epoch_cost_accuracy.append(str(avg_accuracy_list))
            
            print(a_string)
        
        # Writing current epoch data
        with open(logs_path+folder_name+"_data.csv", 'a') as f: # append to the file created
            writer = csv.writer(f, lineterminator="\n")
            writer.writerow(epoch_cost_accuracy)
    
    print("Optimization Finished!")
    end_time = time.time()
    print("Time taken: day, hour, minutes, seconds->", time_taken(start_time, end_time))

In [ ]:
print("Results are saved in the file " + logs_path+folder_name+"_data.csv")